# SOC Copilot Demo: Infrastructure Setup v2

This notebook creates all the infrastructure for the SOC Copilot demo:
- **BigQuery**: Dataset and tables for SOC metrics
- **Firestore**: Collections with seed data (alerts, assets, users, playbooks)
- **Neo4j Aura**: Security graph with ~46 nodes and ~61 relationships
- **TRIGGERED_EVOLUTION**: 3 relationships (THE KEY DIFFERENTIATOR)
- **SUBJECT_TO & MATCHES**: Required relationships for demo flow

## v2 Updates
- ✅ **Colab Secrets support** - Keep passwords out of code!
- ✅ Fixed Gemini model name (`gemini-2.0-flash-001`)
- ✅ Added SUBJECT_TO relationships (Asset → SLA)
- ✅ Added MATCHES relationships (Alert → AttackPattern)

## Secrets to Configure (🔑 icon in left sidebar)
| Secret Name | Example Value |
|-------------|---------------|
| `GCP_PROJECT_ID` | `gen-ai-roi-demo` |
| `NEO4J_URI` | `neo4j+s://a118b6f6.databases.neo4j.io` |
| `NEO4J_PASSWORD` | (your password) |
| `ANTHROPIC_API_KEY` | `sk-ant-...` |

**⏱️ Time to complete:** ~15-20 minutes

---

## Cell 1: Install Required Packages

In [ ]:
# This installs the Python packages we need. Run this first.
# It may take 1-2 minutes.

!pip install --quiet google-cloud-bigquery google-cloud-firestore google-cloud-aiplatform
!pip install --quiet neo4j python-dotenv

print("✓ Packages installed successfully")

## Cell 2: Authenticate with Google Cloud

In [ ]:
# This will open a popup asking you to sign in with your Google account.
# Sign in with the same account that owns your GCP project.

from google.colab import auth
auth.authenticate_user()

print("✓ Authenticated with Google Cloud")

## Cell 2B: Set Up Colab Secrets (RECOMMENDED)

In [ ]:
# Colab Secrets keeps your passwords and API keys secure.
# They're stored encrypted and never visible in your notebook code.
#
# HOW TO SET UP:
# 1. Look at the LEFT SIDEBAR in Colab
# 2. Click the 🔑 KEY icon (below the folder icon)
# 3. Click "+ Add new secret"
# 4. Add these secrets (name → value):
#
#    GCP_PROJECT_ID     → your GCP project ID (e.g., "gen-ai-roi-demo")
#    NEO4J_URI          → your Neo4j URI (e.g., "neo4j+s://a118b6f6.databases.neo4j.io")
#    NEO4J_PASSWORD     → your Neo4j password
#    ANTHROPIC_API_KEY  → your Anthropic API key (sk-ant-...)
#
# 5. Toggle "Notebook access" ON for each secret
# 6. The cells below will automatically load these values!
#
# If you prefer NOT to use Colab Secrets, just edit the fallback values
# in Cells 3 and 21 below.

print("📝 Colab Secrets Setup Checklist:")
print("   [ ] GCP_PROJECT_ID")
print("   [ ] NEO4J_URI")
print("   [ ] NEO4J_PASSWORD")
print("   [ ] ANTHROPIC_API_KEY (optional, for later)")
print()
print("Click the 🔑 key icon in the left sidebar to add these secrets.")

## Cell 3: Set Your Project Configuration

In [ ]:
# This cell tries to load credentials from Colab Secrets first.
# If not found, it falls back to the manual values below.
#
# TO USE COLAB SECRETS (Recommended):
# 1. Click the 🔑 key icon in the left sidebar
# 2. Add these secrets:
#    - GCP_PROJECT_ID: your GCP project ID (e.g., "gen-ai-roi-demo")
#
# OR: Just edit the fallback values below.

import os

# Try to load from Colab Secrets first
try:
    from google.colab import userdata
    PROJECT_ID = userdata.get('GCP_PROJECT_ID')
    print("✓ Loaded PROJECT_ID from Colab Secrets")
except (ImportError, Exception) as e:
    # Fallback to manual configuration
    PROJECT_ID = "soc-copilot-demo"  # ← CHANGE THIS if not using Colab Secrets
    print("ℹ Using manual PROJECT_ID configuration")

REGION = "us-central1"
BIGQUERY_DATASET = "soc"

# Set environment variables
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

print(f"✓ Configuration set:")
print(f"  Project ID: {PROJECT_ID}")
print(f"  Region: {REGION}")
print(f"  BigQuery Dataset: {BIGQUERY_DATASET}")

## Cell 4: Enable Required GCP APIs

In [ ]:
# This enables all the APIs we need. It may take 2-3 minutes.
# You'll see output for each API being enabled.

print("Enabling required APIs... (this takes 2-3 minutes)")
print("-" * 50)

apis = [
    "bigquery.googleapis.com",
    "firestore.googleapis.com", 
    "aiplatform.googleapis.com",
]

for api in apis:
    print(f"Enabling {api}...")
    !gcloud services enable {api} --project={PROJECT_ID}

print("-" * 50)
print("✓ All APIs enabled")

## Cell 5: Initialize Clients

In [ ]:
# This creates the Python clients for each service.

from google.cloud import bigquery
from google.cloud import firestore
import vertexai

# Initialize clients
bq_client = bigquery.Client(project=PROJECT_ID)
db = firestore.Client(project=PROJECT_ID)
vertexai.init(project=PROJECT_ID, location=REGION)

print("✓ Clients initialized:")
print(f"  BigQuery: {PROJECT_ID}")
print(f"  Firestore: {PROJECT_ID}")
print(f"  Vertex AI: {PROJECT_ID} ({REGION})")

## Cell 6: Create BigQuery Dataset

In [ ]:
# This creates the 'soc' dataset in BigQuery.

print(f"Creating BigQuery dataset '{BIGQUERY_DATASET}'...")

dataset_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"

try:
    dataset = bq_client.create_dataset(dataset, exists_ok=True)
    print(f"✓ Dataset '{BIGQUERY_DATASET}' created (or already exists)")
except Exception as e:
    print(f"Error: {e}")

## Cell 7: Create BigQuery Tables

In [ ]:
# Creates the tables for SOC metrics, detection rules, and sprawl detection.

print("Creating BigQuery tables...")
print("-" * 50)

# Table 1: Metric Contracts (SOC KPIs)
metric_schema = [
    bigquery.SchemaField("metric_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("version", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("definition", "STRING"),
    bigquery.SchemaField("formula", "STRING"),
    bigquery.SchemaField("owner_email", "STRING"),
    bigquery.SchemaField("grain", "STRING", mode="REPEATED"),
    bigquery.SchemaField("source_tables", "STRING", mode="REPEATED"),
    bigquery.SchemaField("join_logic", "STRING"),
    bigquery.SchemaField("filters_allowed", "STRING", mode="REPEATED"),
    bigquery.SchemaField("freshness_sla_hours", "INTEGER"),
    bigquery.SchemaField("quality_threshold", "FLOAT"),
    bigquery.SchemaField("status", "STRING"),
    bigquery.SchemaField("deprecated_by", "STRING"),
    bigquery.SchemaField("created_at", "TIMESTAMP"),
    bigquery.SchemaField("updated_at", "TIMESTAMP"),
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.metric_contracts"
table = bigquery.Table(table_id, schema=metric_schema)
try:
    table = bq_client.create_table(table, exists_ok=True)
    print(f"✓ Created table: metric_contracts")
except Exception as e:
    print(f"  Note: {e}")

# Table 2: SOC Metrics (time-series data)
soc_metrics_schema = [
    bigquery.SchemaField("date", "DATE", mode="REQUIRED"),
    bigquery.SchemaField("severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("alert_type", "STRING"),
    bigquery.SchemaField("alerts_received", "INTEGER"),
    bigquery.SchemaField("alerts_auto_closed", "INTEGER"),
    bigquery.SchemaField("alerts_escalated_tier2", "INTEGER"),
    bigquery.SchemaField("alerts_escalated_ir", "INTEGER"),
    bigquery.SchemaField("avg_mttr_minutes", "FLOAT"),
    bigquery.SchemaField("avg_mttd_minutes", "FLOAT"),
    bigquery.SchemaField("fp_count", "INTEGER"),
    bigquery.SchemaField("tp_count", "INTEGER"),
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.soc_metrics"
table = bigquery.Table(table_id, schema=soc_metrics_schema)
try:
    table = bq_client.create_table(table, exists_ok=True)
    print(f"✓ Created table: soc_metrics")
except Exception as e:
    print(f"  Note: {e}")

# Table 3: Detection Rules Registry
detection_rules_schema = [
    bigquery.SchemaField("rule_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("description", "STRING"),
    bigquery.SchemaField("mitre_technique", "STRING"),
    bigquery.SchemaField("severity", "STRING"),
    bigquery.SchemaField("alerts_last_30d", "INTEGER"),
    bigquery.SchemaField("fp_rate", "FLOAT"),
    bigquery.SchemaField("status", "STRING"),
    bigquery.SchemaField("created_at", "TIMESTAMP"),
    bigquery.SchemaField("updated_at", "TIMESTAMP"),
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.detection_rules"
table = bigquery.Table(table_id, schema=detection_rules_schema)
try:
    table = bq_client.create_table(table, exists_ok=True)
    print(f"✓ Created table: detection_rules")
except Exception as e:
    print(f"  Note: {e}")

# Table 4: Rule Sprawl Registry (for duplicate detection)
sprawl_schema = [
    bigquery.SchemaField("rule_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("similar_rule_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("similarity_score", "FLOAT"),
    bigquery.SchemaField("pipeline_count", "INTEGER"),
    bigquery.SchemaField("monthly_alert_impact", "INTEGER"),
    bigquery.SchemaField("estimated_monthly_cost", "FLOAT"),
    bigquery.SchemaField("status", "STRING"),
    bigquery.SchemaField("detected_at", "TIMESTAMP"),
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.rule_sprawl_registry"
table = bigquery.Table(table_id, schema=sprawl_schema)
try:
    table = bq_client.create_table(table, exists_ok=True)
    print(f"✓ Created table: rule_sprawl_registry")
except Exception as e:
    print(f"  Note: {e}")

print("-" * 50)
print("✓ All BigQuery tables created successfully")

## Cell 8: Seed BigQuery with SOC Metric Contracts

In [ ]:
# This populates the metric_contracts table with SOC-specific KPIs.

from datetime import datetime, timedelta
import json

print("Seeding SOC metric contracts...")
print("-" * 50)

metric_contracts = [
    {
        "metric_id": "mttr_by_severity",
        "version": 2,
        "name": "MTTR by Severity",
        "definition": "Mean Time to Respond - average minutes from alert creation to first analyst action, grouped by severity level.",
        "formula": "AVG(TIMESTAMP_DIFF(first_response_time, alert_created_time, MINUTE))",
        "owner_email": "soc_analytics@company.com",
        "grain": ["severity", "week_start"],
        "source_tables": ["soc.alerts", "soc.analyst_actions"],
        "join_logic": "alerts.alert_id = analyst_actions.alert_id",
        "filters_allowed": ["severity", "alert_type", "analyst_id", "shift"],
        "freshness_sla_hours": 1,
        "quality_threshold": 0.98,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-08-15T00:00:00Z",
        "updated_at": "2026-01-20T00:00:00Z",
    },
    {
        "metric_id": "mttd_by_source",
        "version": 1,
        "name": "MTTD by Source",
        "definition": "Mean Time to Detect - average minutes from event occurrence to alert generation, by detection source.",
        "formula": "AVG(TIMESTAMP_DIFF(alert_created_time, event_time, MINUTE))",
        "owner_email": "soc_analytics@company.com",
        "grain": ["source", "day"],
        "source_tables": ["soc.alerts", "soc.raw_events"],
        "join_logic": "alerts.event_id = raw_events.event_id",
        "filters_allowed": ["source", "severity", "alert_type"],
        "freshness_sla_hours": 4,
        "quality_threshold": 0.95,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-09-01T00:00:00Z",
        "updated_at": "2026-01-18T00:00:00Z",
    },
    {
        "metric_id": "auto_close_rate",
        "version": 2,
        "name": "Auto-Close Rate",
        "definition": "Percentage of alerts automatically closed by the SOC Copilot without human intervention.",
        "formula": "COUNT(CASE WHEN closed_by = 'copilot' THEN 1 END) / COUNT(*) * 100",
        "owner_email": "soc_analytics@company.com",
        "grain": ["week", "alert_type"],
        "source_tables": ["soc.alerts", "soc.resolutions"],
        "join_logic": "alerts.alert_id = resolutions.alert_id",
        "filters_allowed": ["alert_type", "severity", "pattern_matched"],
        "freshness_sla_hours": 1,
        "quality_threshold": 0.99,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-10-01T00:00:00Z",
        "updated_at": "2026-01-25T00:00:00Z",
    },
    {
        "metric_id": "fp_rate_by_rule",
        "version": 1,
        "name": "False Positive Rate by Rule",
        "definition": "Percentage of alerts from each detection rule that were confirmed as false positives.",
        "formula": "COUNT(CASE WHEN resolution = 'false_positive' THEN 1 END) / COUNT(*) * 100",
        "owner_email": "detection_engineering@company.com",
        "grain": ["rule_id", "week"],
        "source_tables": ["soc.alerts", "soc.resolutions"],
        "join_logic": "alerts.alert_id = resolutions.alert_id",
        "filters_allowed": ["rule_id", "severity", "mitre_technique"],
        "freshness_sla_hours": 24,
        "quality_threshold": 0.95,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-07-15T00:00:00Z",
        "updated_at": "2026-01-15T00:00:00Z",
    },
    {
        "metric_id": "escalation_rate",
        "version": 1,
        "name": "Escalation Rate",
        "definition": "Percentage of alerts escalated to Tier 2 or Incident Response team.",
        "formula": "COUNT(CASE WHEN escalated = true THEN 1 END) / COUNT(*) * 100",
        "owner_email": "soc_analytics@company.com",
        "grain": ["week", "severity"],
        "source_tables": ["soc.alerts", "soc.escalations"],
        "join_logic": "alerts.alert_id = escalations.alert_id",
        "filters_allowed": ["severity", "alert_type", "escalation_reason"],
        "freshness_sla_hours": 4,
        "quality_threshold": 0.98,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-08-01T00:00:00Z",
        "updated_at": "2026-01-10T00:00:00Z",
    },
    {
        "metric_id": "analyst_efficiency",
        "version": 1,
        "name": "Analyst Efficiency",
        "definition": "Average number of alerts resolved per analyst per shift.",
        "formula": "COUNT(resolved_alerts) / COUNT(DISTINCT analyst_id || shift_date)",
        "owner_email": "soc_management@company.com",
        "grain": ["analyst_id", "shift_date"],
        "source_tables": ["soc.resolutions", "soc.analyst_shifts"],
        "join_logic": "resolutions.analyst_id = analyst_shifts.analyst_id",
        "filters_allowed": ["analyst_id", "shift", "alert_type"],
        "freshness_sla_hours": 24,
        "quality_threshold": 0.95,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-09-15T00:00:00Z",
        "updated_at": "2026-01-20T00:00:00Z",
    },
    {
        "metric_id": "sla_compliance",
        "version": 2,
        "name": "SLA Compliance Rate",
        "definition": "Percentage of alerts responded to within SLA timeframes by severity.",
        "formula": "COUNT(CASE WHEN response_time <= sla_target THEN 1 END) / COUNT(*) * 100",
        "owner_email": "soc_management@company.com",
        "grain": ["severity", "week"],
        "source_tables": ["soc.alerts", "soc.sla_targets", "soc.resolutions"],
        "join_logic": "alerts.severity = sla_targets.severity AND alerts.alert_id = resolutions.alert_id",
        "filters_allowed": ["severity", "shift", "alert_type"],
        "freshness_sla_hours": 4,
        "quality_threshold": 0.99,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-06-01T00:00:00Z",
        "updated_at": "2026-01-22T00:00:00Z",
    },
    {
        "metric_id": "pattern_match_rate",
        "version": 1,
        "name": "Pattern Match Rate",
        "definition": "Percentage of alerts that matched a known attack pattern.",
        "formula": "COUNT(CASE WHEN pattern_id IS NOT NULL THEN 1 END) / COUNT(*) * 100",
        "owner_email": "soc_analytics@company.com",
        "grain": ["week", "alert_type"],
        "source_tables": ["soc.alerts", "soc.pattern_matches"],
        "join_logic": "alerts.alert_id = pattern_matches.alert_id",
        "filters_allowed": ["alert_type", "pattern_id"],
        "freshness_sla_hours": 1,
        "quality_threshold": 0.95,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-11-01T00:00:00Z",
        "updated_at": "2026-01-18T00:00:00Z",
    },
    {
        "metric_id": "mttr_by_severity_legacy",
        "version": 1,
        "name": "MTTR by Severity (Legacy)",
        "definition": "Old MTTR calculation using different timestamp fields.",
        "formula": "AVG(close_time - open_time)",
        "owner_email": "legacy_soc@company.com",
        "grain": ["severity"],
        "source_tables": ["legacy.tickets"],
        "join_logic": None,
        "filters_allowed": ["severity"],
        "freshness_sla_hours": 48,
        "quality_threshold": 0.85,
        "status": "deprecated",
        "deprecated_by": "mttr_by_severity",
        "created_at": "2023-01-15T00:00:00Z",
        "updated_at": "2025-06-01T00:00:00Z",
    },
    {
        "metric_id": "threat_coverage",
        "version": 1,
        "name": "MITRE ATT&CK Coverage",
        "definition": "Percentage of MITRE ATT&CK techniques covered by active detection rules.",
        "formula": "COUNT(DISTINCT covered_techniques) / COUNT(DISTINCT all_techniques) * 100",
        "owner_email": "detection_engineering@company.com",
        "grain": ["tactic", "month"],
        "source_tables": ["soc.detection_rules", "mitre.techniques"],
        "join_logic": "detection_rules.mitre_technique = techniques.technique_id",
        "filters_allowed": ["tactic", "platform"],
        "freshness_sla_hours": 168,
        "quality_threshold": 0.90,
        "status": "active",
        "deprecated_by": None,
        "created_at": "2025-05-01T00:00:00Z",
        "updated_at": "2026-01-05T00:00:00Z",
    },
]

# Insert into BigQuery
table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.metric_contracts"

for contract in metric_contracts:
    contract["grain"] = json.dumps(contract["grain"]) if isinstance(contract["grain"], list) else contract["grain"]
    contract["source_tables"] = json.dumps(contract["source_tables"]) if isinstance(contract["source_tables"], list) else contract["source_tables"]
    contract["filters_allowed"] = json.dumps(contract["filters_allowed"]) if isinstance(contract["filters_allowed"], list) else contract["filters_allowed"]

rows_to_insert = []
for c in metric_contracts:
    row = {
        "metric_id": c["metric_id"],
        "version": c["version"],
        "name": c["name"],
        "definition": c["definition"],
        "formula": c["formula"],
        "owner_email": c["owner_email"],
        "grain": json.loads(c["grain"]) if isinstance(c["grain"], str) else c["grain"],
        "source_tables": json.loads(c["source_tables"]) if isinstance(c["source_tables"], str) else c["source_tables"],
        "join_logic": c["join_logic"],
        "filters_allowed": json.loads(c["filters_allowed"]) if isinstance(c["filters_allowed"], str) else c["filters_allowed"],
        "freshness_sla_hours": c["freshness_sla_hours"],
        "quality_threshold": c["quality_threshold"],
        "status": c["status"],
        "deprecated_by": c["deprecated_by"],
        "created_at": c["created_at"],
        "updated_at": c["updated_at"],
    }
    rows_to_insert.append(row)

errors = bq_client.insert_rows_json(table_id, rows_to_insert)
if errors:
    print(f"Errors: {errors}")
else:
    print(f"✓ Inserted {len(rows_to_insert)} metric contracts")

print("-" * 50)
print("✓ SOC metric contracts seeded")

## Cell 9: Seed BigQuery with SOC Metrics (Time Series)

In [ ]:
# This creates 4 weeks of realistic SOC metrics data.

print("Seeding SOC metrics (4 weeks of data)...")
print("-" * 50)

from datetime import datetime, timedelta
import random

# Generate 4 weeks of data
base_date = datetime(2026, 1, 6)  # Start of first week
soc_metrics_data = []

severities = ["critical", "high", "medium", "low"]
alert_types = ["anomalous_login", "phishing", "malware_detection", "data_exfiltration"]

# Week 1: Lower auto-close, higher MTTR (baseline)
# Week 4: Higher auto-close, lower MTTR (improvement)
for week in range(4):
    week_start = base_date + timedelta(weeks=week)
    
    # Calculate improvement factors
    improvement = week * 0.07  # 7% improvement per week
    
    for day_offset in range(7):
        current_date = week_start + timedelta(days=day_offset)
        
        for severity in severities:
            for alert_type in alert_types:
                # Base values vary by severity
                base_alerts = {"critical": 15, "high": 45, "medium": 120, "low": 200}[severity]
                base_auto_close = {"critical": 0.3, "high": 0.5, "medium": 0.7, "low": 0.85}[severity]
                base_mttr = {"critical": 8, "high": 15, "medium": 45, "low": 120}[severity]
                
                # Add randomness and improvement
                alerts = int(base_alerts * (0.8 + random.random() * 0.4))
                auto_close_rate = min(0.95, base_auto_close + improvement + random.random() * 0.05)
                auto_closed = int(alerts * auto_close_rate)
                
                escalated_tier2 = int((alerts - auto_closed) * 0.6)
                escalated_ir = int((alerts - auto_closed) * 0.3)
                
                mttr = max(2, base_mttr * (1 - improvement * 0.5) + random.random() * 5)
                mttd = max(1, 3 + random.random() * 2)
                
                fp_count = int(auto_closed * (0.8 + random.random() * 0.15))
                tp_count = alerts - fp_count
                
                soc_metrics_data.append({
                    "date": current_date.strftime("%Y-%m-%d"),
                    "severity": severity,
                    "alert_type": alert_type,
                    "alerts_received": alerts,
                    "alerts_auto_closed": auto_closed,
                    "alerts_escalated_tier2": escalated_tier2,
                    "alerts_escalated_ir": escalated_ir,
                    "avg_mttr_minutes": round(mttr, 1),
                    "avg_mttd_minutes": round(mttd, 1),
                    "fp_count": fp_count,
                    "tp_count": tp_count,
                })

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.soc_metrics"
errors = bq_client.insert_rows_json(table_id, soc_metrics_data)
if errors:
    print(f"Errors: {errors[:3]}")  # Show first 3 errors
else:
    print(f"✓ Inserted {len(soc_metrics_data)} SOC metric rows (4 weeks)")

print("-" * 50)
print("✓ SOC metrics seeded")

## Cell 10: Seed BigQuery with Detection Rules

In [ ]:
# This creates sample detection rules for the demo.

print("Seeding detection rules registry...")
print("-" * 50)

detection_rules = [
    {
        "rule_id": "anomalous_login_v2",
        "name": "Anomalous Login Detection v2",
        "description": "Detects logins from unusual locations, times, or devices",
        "mitre_technique": "T1078",
        "severity": "medium",
        "alerts_last_30d": 4523,
        "fp_rate": 0.72,
        "status": "active",
        "created_at": "2025-03-15T00:00:00Z",
        "updated_at": "2026-01-20T00:00:00Z",
    },
    {
        "rule_id": "anomalous_login_legacy",
        "name": "Anomalous Login (Legacy)",
        "description": "Old version - uses IP geolocation only",
        "mitre_technique": "T1078",
        "severity": "medium",
        "alerts_last_30d": 2400,
        "fp_rate": 0.89,
        "status": "deprecated",
        "created_at": "2023-06-01T00:00:00Z",
        "updated_at": "2025-12-01T00:00:00Z",
    },
    {
        "rule_id": "phishing_email_v3",
        "name": "Phishing Email Detection v3",
        "description": "Identifies phishing attempts using URL analysis and sender reputation",
        "mitre_technique": "T1566.001",
        "severity": "high",
        "alerts_last_30d": 1876,
        "fp_rate": 0.15,
        "status": "active",
        "created_at": "2025-08-01T00:00:00Z",
        "updated_at": "2026-01-18T00:00:00Z",
    },
    {
        "rule_id": "malware_hash_match",
        "name": "Known Malware Hash Detection",
        "description": "Matches file hashes against threat intelligence feeds",
        "mitre_technique": "T1204",
        "severity": "critical",
        "alerts_last_30d": 234,
        "fp_rate": 0.02,
        "status": "active",
        "created_at": "2024-01-15T00:00:00Z",
        "updated_at": "2026-01-25T00:00:00Z",
    },
    {
        "rule_id": "dlp_sensitive_data",
        "name": "Sensitive Data Exfiltration",
        "description": "Detects potential data exfiltration based on volume and destination",
        "mitre_technique": "T1048",
        "severity": "high",
        "alerts_last_30d": 456,
        "fp_rate": 0.35,
        "status": "active",
        "created_at": "2025-04-01T00:00:00Z",
        "updated_at": "2026-01-15T00:00:00Z",
    },
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.detection_rules"
errors = bq_client.insert_rows_json(table_id, detection_rules)
if errors:
    print(f"Errors: {errors}")
else:
    print(f"✓ Inserted {len(detection_rules)} detection rules")

print("-" * 50)
print("✓ Detection rules seeded")

## Cell 11: Seed BigQuery with Rule Sprawl Data

In [ ]:
# This creates the sprawl detection data showing duplicate rules.

print("Seeding rule sprawl registry...")
print("-" * 50)

rule_sprawl = [
    {
        "rule_id": "anomalous_login_legacy",
        "similar_rule_id": "anomalous_login_v2",
        "similarity_score": 0.87,
        "pipeline_count": 3,
        "monthly_alert_impact": 2400,
        "estimated_monthly_cost": 18000.0,
        "status": "reviewing",
        "detected_at": "2026-01-15T00:00:00Z",
    },
]

table_id = f"{PROJECT_ID}.{BIGQUERY_DATASET}.rule_sprawl_registry"
errors = bq_client.insert_rows_json(table_id, rule_sprawl)
if errors:
    print(f"Errors: {errors}")
else:
    print(f"✓ Inserted {len(rule_sprawl)} sprawl detection entries")

print("-" * 50)
print("✓ Rule sprawl data seeded")

## Cell 12: Create Firestore Collections - Assets

In [ ]:
# This creates the assets (devices/systems) for the security demo.

print("Creating Firestore assets collection...")
print("-" * 50)

assets = [
    {
        "asset_id": "LAPTOP-JSMITH",
        "hostname": "LAPTOP-JSMITH",
        "type": "endpoint",
        "os": "Windows 11 Enterprise",
        "criticality": "high",
        "business_unit": "Finance",
        "owner_id": "jsmith@company.com",
        "last_seen": "2026-01-31T10:30:00Z",
    },
    {
        "asset_id": "SRV-DB-PROD-01",
        "hostname": "SRV-DB-PROD-01",
        "type": "server",
        "os": "RHEL 8.6",
        "criticality": "critical",
        "business_unit": "IT",
        "owner_id": "db_admin@company.com",
        "last_seen": "2026-01-31T10:35:00Z",
    },
    {
        "asset_id": "LAPTOP-MCHEN",
        "hostname": "LAPTOP-MCHEN",
        "type": "endpoint",
        "os": "macOS Sonoma",
        "criticality": "high",
        "business_unit": "Engineering",
        "owner_id": "mchen@company.com",
        "last_seen": "2026-01-31T09:15:00Z",
    },
    {
        "asset_id": "LAPTOP-AGARCIA",
        "hostname": "LAPTOP-AGARCIA",
        "type": "endpoint",
        "os": "macOS Sonoma",
        "criticality": "medium",
        "business_unit": "Engineering",
        "owner_id": "agarcia@company.com",
        "last_seen": "2026-01-31T08:45:00Z",
    },
    {
        "asset_id": "MAIL-GW-01",
        "hostname": "MAIL-GW-01",
        "type": "network",
        "os": "Proofpoint Appliance",
        "criticality": "critical",
        "business_unit": "IT",
        "owner_id": "mail_admin@company.com",
        "last_seen": "2026-01-31T10:40:00Z",
    },
]

for asset in assets:
    db.collection("assets").document(asset["asset_id"]).set(asset)
    print(f"  ✓ Created asset: {asset['asset_id']}")

print("-" * 50)
print(f"✓ Created {len(assets)} assets")

## Cell 13: Create Firestore Collections - Users

In [ ]:
# This creates user profiles with risk scores.

print("Creating Firestore users collection...")
print("-" * 50)

users = [
    {
        "user_id": "jsmith@company.com",
        "name": "John Smith",
        "title": "VP Finance",
        "department": "Finance",
        "risk_score": 0.85,
        "is_privileged": True,
        "manager": "cjohnson@company.com",
    },
    {
        "user_id": "mchen@company.com",
        "name": "Mary Chen",
        "title": "Director Engineering",
        "department": "Engineering",
        "risk_score": 0.72,
        "is_privileged": True,
        "manager": "cjohnson@company.com",
    },
    {
        "user_id": "agarcia@company.com",
        "name": "Ana Garcia",
        "title": "Senior Developer",
        "department": "Engineering",
        "risk_score": 0.35,
        "is_privileged": False,
        "manager": "mchen@company.com",
    },
    {
        "user_id": "cjohnson@company.com",
        "name": "Chris Johnson",
        "title": "CEO",
        "department": "Executive",
        "risk_score": 0.95,
        "is_privileged": True,
        "manager": None,
    },
    {
        "user_id": "blee@company.com",
        "name": "Bob Lee",
        "title": "SOC Analyst",
        "department": "Security",
        "risk_score": 0.25,
        "is_privileged": False,
        "manager": "ciso@company.com",
    },
]

for user in users:
    db.collection("users").document(user["user_id"]).set(user)
    print(f"  ✓ Created user: {user['name']}")

print("-" * 50)
print(f"✓ Created {len(users)} users")

## Cell 14: Create Firestore Collections - Playbooks

In [ ]:
# This creates SOC response playbooks.

print("Creating Firestore playbooks collection...")
print("-" * 50)

playbooks = [
    {
        "playbook_id": "PB-LOGIN-001",
        "name": "Anomalous Login Response",
        "description": "Response procedure for anomalous login alerts",
        "alert_types": ["anomalous_login"],
        "steps": [
            "Check user travel status",
            "Verify VPN location",
            "Confirm MFA completion",
            "Check device fingerprint",
            "Review historical patterns"
        ],
        "auto_actions": ["false_positive_close", "enrich_and_wait"],
        "escalation_actions": ["escalate_tier2", "escalate_incident"],
        "sla_minutes": 15,
        "owner": "soc_management@company.com",
    },
    {
        "playbook_id": "PB-PHISH-001",
        "name": "Phishing Response",
        "description": "Response procedure for phishing alerts",
        "alert_types": ["phishing"],
        "steps": [
            "Check sender reputation",
            "Analyze URLs",
            "Check for known campaigns",
            "Quarantine if malicious",
            "Notify affected users"
        ],
        "auto_actions": ["auto_remediate"],
        "escalation_actions": ["escalate_tier2"],
        "sla_minutes": 30,
        "owner": "soc_management@company.com",
    },
    {
        "playbook_id": "PB-MALWARE-001",
        "name": "Malware Response",
        "description": "Response procedure for malware detection",
        "alert_types": ["malware_detection"],
        "steps": [
            "Assess asset criticality",
            "Check for lateral movement",
            "Isolate if confirmed",
            "Collect forensic data",
            "Initiate IR if critical"
        ],
        "auto_actions": ["auto_remediate"],
        "escalation_actions": ["escalate_incident"],
        "sla_minutes": 10,
        "owner": "ir_team@company.com",
    },
    {
        "playbook_id": "PB-DLP-001",
        "name": "Data Exfiltration Response",
        "description": "Response procedure for DLP alerts",
        "alert_types": ["data_exfiltration"],
        "steps": [
            "Identify data classification",
            "Assess volume and destination",
            "Check user authorization",
            "Block if unauthorized",
            "Escalate to legal if PII"
        ],
        "auto_actions": [],
        "escalation_actions": ["escalate_incident"],
        "sla_minutes": 5,
        "owner": "ir_team@company.com",
    },
]

for playbook in playbooks:
    db.collection("playbooks").document(playbook["playbook_id"]).set(playbook)
    print(f"  ✓ Created playbook: {playbook['name']}")

print("-" * 50)
print(f"✓ Created {len(playbooks)} playbooks")

## Cell 15: Create Firestore Collections - Attack Patterns

In [ ]:
# This creates learned attack patterns (the key to compounding).

print("Creating Firestore patterns collection...")
print("-" * 50)

patterns = [
    {
        "pattern_id": "PAT-TRAVEL-001",
        "name": "Travel Login False Positive",
        "description": "Login from travel destination with matching VPN and MFA",
        "alert_types": ["anomalous_login"],
        "match_criteria": {
            "user_traveling": True,
            "vpn_matches_location": True,
            "mfa_completed": True
        },
        "fp_rate": 0.94,
        "occurrence_count": 127,
        "confidence": 0.92,
        "recommended_action": "false_positive_close",
        "created_at": "2025-10-15T00:00:00Z",
        "updated_at": "2026-01-30T00:00:00Z",
    },
    {
        "pattern_id": "PAT-PHISH-KNOWN",
        "name": "Known Phishing Campaign",
        "description": "Email matches known phishing campaign signature",
        "alert_types": ["phishing"],
        "match_criteria": {
            "known_campaign_signature": True
        },
        "fp_rate": 0.02,
        "occurrence_count": 89,
        "confidence": 0.96,
        "recommended_action": "auto_remediate",
        "created_at": "2025-11-01T00:00:00Z",
        "updated_at": "2026-01-28T00:00:00Z",
    },
    {
        "pattern_id": "PAT-MALWARE-ISOLATE",
        "name": "Standard Malware Isolation",
        "description": "Non-critical asset with confirmed malware hash",
        "alert_types": ["malware_detection"],
        "match_criteria": {
            "asset_criticality_not": "critical",
            "hash_confirmed": True
        },
        "fp_rate": 0.08,
        "occurrence_count": 34,
        "confidence": 0.91,
        "recommended_action": "auto_remediate",
        "created_at": "2025-09-01T00:00:00Z",
        "updated_at": "2026-01-25T00:00:00Z",
    },
    {
        "pattern_id": "PAT-VPN-KNOWN",
        "name": "Known VPN Provider",
        "description": "Login from recognized corporate VPN provider",
        "alert_types": ["anomalous_login"],
        "match_criteria": {
            "vpn_provider_known": True,
            "device_fingerprint_match": True
        },
        "fp_rate": 0.96,
        "occurrence_count": 245,
        "confidence": 0.94,
        "recommended_action": "false_positive_close",
        "created_at": "2025-08-15T00:00:00Z",
        "updated_at": "2026-01-29T00:00:00Z",
    },
    {
        "pattern_id": "PAT-LOGIN-NORMAL",
        "name": "Normal Location Login",
        "description": "Login from user's normal location during business hours",
        "alert_types": ["anomalous_login"],
        "match_criteria": {
            "normal_location": True,
            "business_hours": True
        },
        "fp_rate": 0.98,
        "occurrence_count": 2847,
        "confidence": 0.97,
        "recommended_action": "false_positive_close",
        "created_at": "2025-06-01T00:00:00Z",
        "updated_at": "2026-01-30T00:00:00Z",
    },
]

for pattern in patterns:
    db.collection("patterns").document(pattern["pattern_id"]).set(pattern)
    print(f"  ✓ Created pattern: {pattern['name']}")

print("-" * 50)
print(f"✓ Created {len(patterns)} attack patterns")

## Cell 16: Create Firestore Collections - Alerts

In [ ]:
# This creates sample alerts for the demo.

print("Creating Firestore alerts collection...")
print("-" * 50)

alerts = [
    {
        "alert_id": "ALERT-7823",
        "alert_type": "anomalous_login",
        "severity": "medium",
        "source": "Splunk",
        "source_rule_id": "anomalous_login_v2",
        "asset_id": "LAPTOP-JSMITH",
        "asset_hostname": "LAPTOP-JSMITH",
        "user_id": "jsmith@company.com",
        "source_location": "Singapore",
        "source_ip": "103.45.67.89",
        "timestamp": "2026-01-31T03:47:00Z",
        "status": "pending",
        "description": "Login from unusual location: Singapore at 3:47 AM local time",
    },
    {
        "alert_id": "ALERT-7822",
        "alert_type": "phishing",
        "severity": "high",
        "source": "Proofpoint",
        "source_rule_id": "phishing_email_v3",
        "asset_id": "MAIL-GW-01",
        "asset_hostname": "MAIL-GW-01",
        "user_id": "finance_team@company.com",
        "source_location": "External",
        "source_ip": "185.234.72.1",
        "timestamp": "2026-01-31T08:15:00Z",
        "status": "pending",
        "description": "Phishing email detected targeting Finance team",
    },
    {
        "alert_id": "ALERT-7821",
        "alert_type": "malware_detection",
        "severity": "critical",
        "source": "CrowdStrike",
        "source_rule_id": "malware_hash_match",
        "asset_id": "SRV-DB-PROD-01",
        "asset_hostname": "SRV-DB-PROD-01",
        "user_id": "system",
        "source_location": "Internal",
        "source_ip": "10.0.1.50",
        "timestamp": "2026-01-31T09:30:00Z",
        "status": "pending",
        "description": "Known malware hash detected on production database server",
    },
    {
        "alert_id": "ALERT-7820",
        "alert_type": "data_exfiltration",
        "severity": "high",
        "source": "DLP",
        "source_rule_id": "dlp_sensitive_data",
        "asset_id": "LAPTOP-MCHEN",
        "asset_hostname": "LAPTOP-MCHEN",
        "user_id": "mchen@company.com",
        "source_location": "Internal",
        "source_ip": "10.0.5.23",
        "timestamp": "2026-01-31T10:15:00Z",
        "status": "pending",
        "dlp_classification": "confidential",
        "description": "Large data transfer to external cloud storage",
    },
    {
        "alert_id": "ALERT-7819",
        "alert_type": "anomalous_login",
        "severity": "low",
        "source": "Splunk",
        "source_rule_id": "anomalous_login_v2",
        "asset_id": "LAPTOP-AGARCIA",
        "asset_hostname": "LAPTOP-AGARCIA",
        "user_id": "agarcia@company.com",
        "source_location": "Denver",
        "source_ip": "72.45.123.89",
        "timestamp": "2026-01-31T07:30:00Z",
        "status": "pending",
        "description": "Login from Denver (user normally in San Francisco)",
    },
]

for alert in alerts:
    db.collection("alerts").document(alert["alert_id"]).set(alert)
    print(f"  ✓ Created alert: {alert['alert_id']} ({alert['alert_type']})")

print("-" * 50)
print(f"✓ Created {len(alerts)} alerts")

## Cell 17: Create Firestore Collections - Deployments

In [ ]:
# This creates the agent deployment registry (for Tab 2).

print("Creating Firestore deployments collection...")
print("-" * 50)

deployments = [
    {
        "deployment_id": "soc-copilot-v3.1",
        "agent_name": "soc-copilot",
        "version": "v3.1",
        "status": "active",
        "traffic_pct": 90,
        "deployed_at": "2026-01-15T00:00:00Z",
        "config": {
            "auto_close_threshold": 0.88,
            "escalation_threshold": 0.75,
            "model": "rule-based-v3"
        },
        "metrics_7d": {
            "alerts_processed": 4847,
            "auto_close_rate": 0.732,
            "avg_confidence": 0.89,
            "escalation_rate": 0.12,
        }
    },
    {
        "deployment_id": "soc-copilot-v3.2",
        "agent_name": "soc-copilot",
        "version": "v3.2",
        "status": "canary",
        "traffic_pct": 10,
        "deployed_at": "2026-01-28T00:00:00Z",
        "config": {
            "auto_close_threshold": 0.90,
            "escalation_threshold": 0.78,
            "model": "rule-based-v3.2"
        },
        "metrics_7d": {
            "alerts_processed": 253,
            "auto_close_rate": 0.718,
            "avg_confidence": 0.87,
            "escalation_rate": 0.15,
        }
    },
]

for deployment in deployments:
    db.collection("deployments").document(deployment["deployment_id"]).set(deployment)
    print(f"  ✓ Created deployment: {deployment['version']} ({deployment['status']})")

print("-" * 50)
print(f"✓ Created {len(deployments)} deployments")

## Cell 18: Create Firestore Collections - Travel Context

In [ ]:
# This creates travel records for context enrichment.

print("Creating Firestore travel collection...")
print("-" * 50)

travel_records = [
    {
        "travel_id": "TRAVEL-001",
        "user_id": "jsmith@company.com",
        "destination": "Singapore",
        "start_date": "2026-01-28",
        "end_date": "2026-02-05",
        "purpose": "Client meetings",
        "vpn_expected": ["Marriott Hotels", "Singapore Airlines Lounge"],
        "approved_by": "cjohnson@company.com",
    },
    {
        "travel_id": "TRAVEL-002",
        "user_id": "agarcia@company.com",
        "destination": "Denver",
        "start_date": "2026-01-30",
        "end_date": "2026-02-01",
        "purpose": "Team offsite",
        "vpn_expected": ["Hilton Hotels"],
        "approved_by": "mchen@company.com",
    },
]

for travel in travel_records:
    db.collection("travel").document(travel["travel_id"]).set(travel)
    print(f"  ✓ Created travel: {travel['user_id']} → {travel['destination']}")

print("-" * 50)
print(f"✓ Created {len(travel_records)} travel records")

## Cell 19: Create Firestore Collections - Decisions (Sample)

In [ ]:
# This creates sample decision records.

print("Creating Firestore decisions collection...")
print("-" * 50)

decisions = [
    {
        "decision_id": "DEC-7818",
        "alert_id": "ALERT-7818",
        "action": "false_positive_close",
        "confidence": 0.94,
        "pattern_id": "PAT-VPN-KNOWN",
        "reasoning": "User connected via known corporate VPN. Device fingerprint matched.",
        "eval_gate_passed": True,
        "executed_at": "2026-01-30T14:30:00Z",
        "created_by": "soc-copilot-v3.1",
    },
    {
        "decision_id": "DEC-7817",
        "alert_id": "ALERT-7817",
        "action": "auto_remediate",
        "confidence": 0.96,
        "pattern_id": "PAT-PHISH-KNOWN",
        "reasoning": "Email matched known Q4 phishing campaign signature. Auto-quarantined.",
        "eval_gate_passed": True,
        "executed_at": "2026-01-30T11:15:00Z",
        "created_by": "soc-copilot-v3.1",
    },
]

for decision in decisions:
    db.collection("decisions").document(decision["decision_id"]).set(decision)
    print(f"  ✓ Created decision: {decision['decision_id']} ({decision['action']})")

print("-" * 50)
print(f"✓ Created {len(decisions)} sample decisions")

## Cell 20: Test Vertex AI (Gemini) Connection

In [ ]:
# This verifies that Gemini is working for LLM narration.

print("Testing Vertex AI (Gemini) connection...")
print("-" * 50)

from vertexai.generative_models import GenerativeModel

try:
    model = GenerativeModel("gemini-2.0-flash-001")
    response = model.generate_content(
        "Say 'SOC Copilot ready' in exactly 3 words.",
        generation_config={
            "temperature": 0.1,
            "max_output_tokens": 20,
        }
    )
    print(f"✓ Gemini response: {response.text.strip()}")
    print("✓ Vertex AI connection successful")
except Exception as e:
    print(f"✗ Error: {e}")
    print("  Make sure Vertex AI API is enabled and you have quota")

## Cell 21: Configure Neo4j Aura Connection

In [ ]:
# This cell tries to load Neo4j credentials from Colab Secrets first.
# If not found, it falls back to the manual values below.
#
# TO USE COLAB SECRETS (Recommended - keeps passwords out of code):
# 1. Click the 🔑 key icon in the left sidebar
# 2. Add these secrets:
#    - NEO4J_URI: your Neo4j Aura URI (e.g., "neo4j+s://a118b6f6.databases.neo4j.io")
#    - NEO4J_PASSWORD: your Neo4j password
#    - ANTHROPIC_API_KEY: your Anthropic API key (for later use)
#
# OR: Just edit the fallback values below.

# Try to load from Colab Secrets first
try:
    from google.colab import userdata
    NEO4J_URI = userdata.get('NEO4J_URI')
    NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
    NEO4J_USER = "neo4j"  # Usually always 'neo4j' for Aura
    print("✓ Loaded Neo4j credentials from Colab Secrets")
except (ImportError, Exception) as e:
    # Fallback to manual configuration
    NEO4J_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"  # ← CHANGE THIS
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "your-password-here"  # ← CHANGE THIS
    print("ℹ Using manual Neo4j configuration")

print("Neo4j configuration set:")
print(f"  URI: {NEO4J_URI}")
print(f"  User: {NEO4J_USER}")
print(f"  Password: {'*' * len(NEO4J_PASSWORD)}")

## Cell 22: Test Neo4j Connection

In [ ]:
# This verifies the Neo4j connection works.

from neo4j import GraphDatabase

print("Testing Neo4j connection...")
print("-" * 50)

try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        result = session.run("RETURN 'Connection successful!' AS message")
        record = result.single()
        print(f"✓ {record['message']}")
    driver.close()
    print("✓ Neo4j connection verified")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    print("  Check your NEO4J_URI and NEO4J_PASSWORD")

## Cell 23: Clear Existing Neo4j Data (Optional)

In [ ]:
# Run this if you need to reset the Neo4j database.
# WARNING: This deletes all existing nodes and relationships!

CLEAR_DATABASE = False  # Change to True to clear

if CLEAR_DATABASE:
    print("Clearing existing Neo4j data...")
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
    driver.close()
    print("✓ Database cleared")
else:
    print("Skipping database clear (CLEAR_DATABASE = False)")

## Cell 24: Create Neo4j Security Entities

In [ ]:
# This creates the core security entity nodes in Neo4j.

print("Creating Neo4j security entity nodes...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # Create Assets
    for asset in assets:
        session.run("""
            MERGE (a:Asset {id: $id})
            SET a.hostname = $hostname,
                a.type = $type,
                a.os = $os,
                a.criticality = $criticality,
                a.business_unit = $business_unit,
                a.owner_id = $owner_id
        """, id=asset["asset_id"], hostname=asset["hostname"], type=asset["type"],
            os=asset["os"], criticality=asset["criticality"], 
            business_unit=asset["business_unit"], owner_id=asset["owner_id"])
    print(f"  ✓ Created {len(assets)} Asset nodes")
    
    # Create Users
    for user in users:
        session.run("""
            MERGE (u:User {id: $id})
            SET u.name = $name,
                u.title = $title,
                u.department = $department,
                u.risk_score = $risk_score,
                u.is_privileged = $is_privileged
        """, id=user["user_id"], name=user["name"], title=user["title"],
            department=user["department"], risk_score=user["risk_score"],
            is_privileged=user["is_privileged"])
    print(f"  ✓ Created {len(users)} User nodes")
    
    # Create AlertTypes
    alert_types = [
        {"id": "anomalous_login", "name": "Anomalous Login", "severity": "medium", "mitre": "T1078"},
        {"id": "phishing", "name": "Phishing", "severity": "high", "mitre": "T1566"},
        {"id": "malware_detection", "name": "Malware Detection", "severity": "critical", "mitre": "T1204"},
        {"id": "data_exfiltration", "name": "Data Exfiltration", "severity": "high", "mitre": "T1048"},
    ]
    for at in alert_types:
        session.run("""
            MERGE (at:AlertType {id: $id})
            SET at.name = $name,
                at.severity = $severity,
                at.mitre_technique = $mitre
        """, **at)
    print(f"  ✓ Created {len(alert_types)} AlertType nodes")
    
    # Create Attack Patterns
    for pattern in patterns:
        session.run("""
            MERGE (p:AttackPattern {id: $id})
            SET p.name = $name,
                p.fp_rate = $fp_rate,
                p.occurrence_count = $occurrence_count,
                p.confidence = $confidence
        """, id=pattern["pattern_id"], name=pattern["name"],
            fp_rate=pattern["fp_rate"], occurrence_count=pattern["occurrence_count"],
            confidence=pattern["confidence"])
    print(f"  ✓ Created {len(patterns)} AttackPattern nodes")
    
    # Create Playbooks
    for pb in playbooks:
        session.run("""
            MERGE (pb:Playbook {id: $id})
            SET pb.name = $name,
                pb.sla_minutes = $sla_minutes
        """, id=pb["playbook_id"], name=pb["name"], sla_minutes=pb["sla_minutes"])
    print(f"  ✓ Created {len(playbooks)} Playbook nodes")
    
    # Create SLAs
    slas = [
        {"id": "SLA-CRITICAL", "name": "Critical SLA", "response_time_minutes": 10, "severity": "critical"},
        {"id": "SLA-HIGH", "name": "High SLA", "response_time_minutes": 30, "severity": "high"},
        {"id": "SLA-MEDIUM", "name": "Medium SLA", "response_time_minutes": 60, "severity": "medium"},
    ]
    for sla in slas:
        session.run("""
            MERGE (s:SLA {id: $id})
            SET s.name = $name,
                s.response_time_minutes = $response_time_minutes,
                s.severity = $severity
        """, **sla)
    print(f"  ✓ Created {len(slas)} SLA nodes")
    
    # Create Travel Context
    for travel in travel_records:
        session.run("""
            MERGE (t:TravelContext {id: $id})
            SET t.user_id = $user_id,
                t.destination = $destination,
                t.start_date = date($start_date),
                t.end_date = date($end_date)
        """, id=travel["travel_id"], user_id=travel["user_id"],
            destination=travel["destination"], start_date=travel["start_date"],
            end_date=travel["end_date"])
    print(f"  ✓ Created {len(travel_records)} TravelContext nodes")

driver.close()
print("-" * 50)
print("✓ Security entity nodes created")

## Cell 25: Create Neo4j Security Relationships

In [ ]:
# This creates the relationships between security entities.

print("Creating Neo4j security relationships...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # User -[:ASSIGNED_TO]-> Asset
    user_asset_mappings = [
        ("jsmith@company.com", "LAPTOP-JSMITH"),
        ("mchen@company.com", "LAPTOP-MCHEN"),
        ("agarcia@company.com", "LAPTOP-AGARCIA"),
    ]
    for user_id, asset_id in user_asset_mappings:
        session.run("""
            MATCH (u:User {id: $user_id})
            MATCH (a:Asset {id: $asset_id})
            MERGE (u)-[:ASSIGNED_TO]->(a)
        """, user_id=user_id, asset_id=asset_id)
    print(f"  ✓ Created ASSIGNED_TO relationships")
    
    # User -[:HAS_TRAVEL]-> TravelContext
    for travel in travel_records:
        session.run("""
            MATCH (u:User {id: $user_id})
            MATCH (t:TravelContext {id: $travel_id})
            MERGE (u)-[:HAS_TRAVEL]->(t)
        """, user_id=travel["user_id"], travel_id=travel["travel_id"])
    print(f"  ✓ Created HAS_TRAVEL relationships")
    
    # AlertType -[:HANDLED_BY]-> Playbook
    alert_playbook_mappings = [
        ("anomalous_login", "PB-LOGIN-001"),
        ("phishing", "PB-PHISH-001"),
        ("malware_detection", "PB-MALWARE-001"),
        ("data_exfiltration", "PB-DLP-001"),
    ]
    for alert_type, playbook_id in alert_playbook_mappings:
        session.run("""
            MATCH (at:AlertType {id: $alert_type})
            MATCH (pb:Playbook {id: $playbook_id})
            MERGE (at)-[:HANDLED_BY]->(pb)
        """, alert_type=alert_type, playbook_id=playbook_id)
    print(f"  ✓ Created HANDLED_BY relationships")

driver.close()
print("-" * 50)
print("✓ Security relationships created")

## Cell 25B: Create Additional Relationships (CRITICAL FOR DEMO)

In [ ]:
# These relationships are required for the demo to work correctly:
# - Asset -[:SUBJECT_TO]-> SLA (for SLA compliance checks)
# - Alert -[:MATCHES]-> AttackPattern (for pattern matching)

print("Creating additional relationships (critical for demo)...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # Asset -[:SUBJECT_TO]-> SLA (based on asset criticality)
    asset_sla_mappings = [
        ("LAPTOP-JSMITH", "SLA-HIGH"),      # High criticality asset
        ("SRV-DB-PROD", "SLA-CRITICAL"),    # Critical server
        ("LAPTOP-MCHEN", "SLA-HIGH"),       # High criticality asset
        ("LAPTOP-AGARCIA", "SLA-MEDIUM"),   # Medium criticality asset
        ("MAIL-GW", "SLA-CRITICAL"),        # Critical infrastructure
    ]
    for asset_id, sla_id in asset_sla_mappings:
        session.run("""
            MATCH (a:Asset {id: $asset_id})
            MATCH (s:SLA {id: $sla_id})
            MERGE (a)-[:SUBJECT_TO]->(s)
        """, asset_id=asset_id, sla_id=sla_id)
    print(f"  ✓ Created {len(asset_sla_mappings)} SUBJECT_TO relationships")
    
    # Alert -[:MATCHES]-> AttackPattern (for known pattern matching)
    alert_pattern_mappings = [
        ("ALERT-7823", "PAT-TRAVEL-001"),   # John Smith travel - matches travel pattern
        ("ALERT-7822", "PAT-MALWARE-ISOLATE"),  # Malware detection
        ("ALERT-7821", "PAT-PHISH-KNOWN"),  # Phishing email
        ("ALERT-7820", "PAT-VPN-KNOWN"),    # VPN login
        ("ALERT-7819", "PAT-LOGIN-NORMAL"), # Normal login pattern
    ]
    for alert_id, pattern_id in alert_pattern_mappings:
        session.run("""
            MATCH (alert:Alert {id: $alert_id})
            MATCH (p:AttackPattern {id: $pattern_id})
            MERGE (alert)-[:MATCHES]->(p)
        """, alert_id=alert_id, pattern_id=pattern_id)
    print(f"  ✓ Created {len(alert_pattern_mappings)} MATCHES relationships")

driver.close()
print("-" * 50)
print("✓ Additional relationships created")

## Cell 26: Create Neo4j Alert Nodes

In [ ]:
# This creates alert nodes and their relationships.

print("Creating Neo4j alert nodes and relationships...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    for alert in alerts:
        # Create Alert node
        session.run("""
            MERGE (a:Alert {id: $id})
            SET a.alert_type = $alert_type,
                a.severity = $severity,
                a.source = $source,
                a.source_location = $source_location,
                a.timestamp = datetime($timestamp),
                a.status = $status,
                a.description = $description
        """, id=alert["alert_id"], alert_type=alert["alert_type"],
            severity=alert["severity"], source=alert["source"],
            source_location=alert["source_location"], timestamp=alert["timestamp"],
            status=alert["status"], description=alert["description"])
        
        # Alert -[:DETECTED_ON]-> Asset
        session.run("""
            MATCH (alert:Alert {id: $alert_id})
            MATCH (asset:Asset {id: $asset_id})
            MERGE (alert)-[:DETECTED_ON]->(asset)
        """, alert_id=alert["alert_id"], asset_id=alert["asset_id"])
        
        # Alert -[:INVOLVES]-> User
        session.run("""
            MATCH (alert:Alert {id: $alert_id})
            MATCH (user:User {id: $user_id})
            MERGE (alert)-[:INVOLVES]->(user)
        """, alert_id=alert["alert_id"], user_id=alert["user_id"])
        
        # Alert -[:CLASSIFIED_AS]-> AlertType
        session.run("""
            MATCH (alert:Alert {id: $alert_id})
            MATCH (at:AlertType {id: $alert_type})
            MERGE (alert)-[:CLASSIFIED_AS]->(at)
        """, alert_id=alert["alert_id"], alert_type=alert["alert_type"])
        
    print(f"  ✓ Created {len(alerts)} Alert nodes with relationships")

driver.close()
print("-" * 50)
print("✓ Alert nodes and relationships created")

## Cell 27: Create Decision Trace Nodes (THE KEY DIFFERENTIATOR)

In [ ]:
# This creates the Decision Trace schema that makes our demo unique.
# - Decision nodes (5)
# - DecisionContext nodes (5)
# - EvolutionEvent nodes (3)

print("Creating Decision Trace nodes (THE KEY DIFFERENTIATOR)...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # Create Decision nodes
    decision_nodes = [
        {
            "id": "DEC-7823",
            "type": "alert_triage",
            "reasoning": "User has active travel to Singapore. VPN from Marriott Hotels matches known provider. MFA completed. Pattern PAT-TRAVEL-001 matched with 127 similar cases.",
            "confidence": 0.92,
            "timestamp": "2026-01-31T03:48:00Z",
            "alert_id": "ALERT-7823",
            "action_taken": "false_positive_close"
        },
        {
            "id": "DEC-7822",
            "type": "alert_triage",
            "reasoning": "Email matches known Q4 phishing campaign (CAMP-2026-Q1-003). URL analysis confirmed malicious. Auto-quarantine executed.",
            "confidence": 0.96,
            "timestamp": "2026-01-31T08:16:00Z",
            "alert_id": "ALERT-7822",
            "action_taken": "auto_remediate"
        },
        {
            "id": "DEC-7821",
            "type": "alert_triage",
            "reasoning": "Malware detected on CRITICAL production database server. Immediate IR escalation required per playbook PB-MALWARE-001.",
            "confidence": 0.96,
            "timestamp": "2026-01-31T09:31:00Z",
            "alert_id": "ALERT-7821",
            "action_taken": "escalate_incident"
        },
        {
            "id": "DEC-7820",
            "type": "alert_triage",
            "reasoning": "Potential data exfiltration from privileged user. Confidential data classification. Escalating to IR per policy.",
            "confidence": 0.97,
            "timestamp": "2026-01-31T10:16:00Z",
            "alert_id": "ALERT-7820",
            "action_taken": "escalate_incident"
        },
        {
            "id": "DEC-7819",
            "type": "alert_triage",
            "reasoning": "User Ana Garcia has active travel to Denver. Login location matches travel destination. Low risk user.",
            "confidence": 0.91,
            "timestamp": "2026-01-31T07:31:00Z",
            "alert_id": "ALERT-7819",
            "action_taken": "false_positive_close"
        },
    ]
    
    for dec in decision_nodes:
        session.run("""
            MERGE (d:Decision {id: $id})
            SET d.type = $type,
                d.reasoning = $reasoning,
                d.confidence = $confidence,
                d.timestamp = datetime($timestamp),
                d.alert_id = $alert_id,
                d.action_taken = $action_taken
        """, **dec)
    print(f"  ✓ Created {len(decision_nodes)} Decision nodes")
    
    # Create DecisionContext nodes
    context_nodes = [
        {
            "id": "CTX-7823",
            "decision_id": "DEC-7823",
            "user_snapshot": '{"name": "John Smith", "title": "VP Finance", "risk_score": 0.85}',
            "asset_snapshot": '{"hostname": "LAPTOP-JSMITH", "criticality": "high"}',
            "patterns_matched": ["PAT-TRAVEL-001"],
            "nodes_consulted": 47
        },
        {
            "id": "CTX-7822",
            "decision_id": "DEC-7822",
            "user_snapshot": '{"name": "Finance Team", "department": "Finance"}',
            "asset_snapshot": '{"hostname": "MAIL-GW-01", "criticality": "critical"}',
            "patterns_matched": ["PAT-PHISH-KNOWN"],
            "nodes_consulted": 23
        },
        {
            "id": "CTX-7821",
            "decision_id": "DEC-7821",
            "user_snapshot": '{"name": "system", "type": "service_account"}',
            "asset_snapshot": '{"hostname": "SRV-DB-PROD-01", "criticality": "critical"}',
            "patterns_matched": [],
            "nodes_consulted": 31
        },
        {
            "id": "CTX-7820",
            "decision_id": "DEC-7820",
            "user_snapshot": '{"name": "Mary Chen", "title": "Director Engineering", "risk_score": 0.72}',
            "asset_snapshot": '{"hostname": "LAPTOP-MCHEN", "criticality": "high"}',
            "patterns_matched": [],
            "nodes_consulted": 28
        },
        {
            "id": "CTX-7819",
            "decision_id": "DEC-7819",
            "user_snapshot": '{"name": "Ana Garcia", "title": "Senior Developer", "risk_score": 0.35}',
            "asset_snapshot": '{"hostname": "LAPTOP-AGARCIA", "criticality": "medium"}',
            "patterns_matched": ["PAT-TRAVEL-001"],
            "nodes_consulted": 35
        },
    ]
    
    for ctx in context_nodes:
        session.run("""
            MERGE (c:DecisionContext {id: $id})
            SET c.decision_id = $decision_id,
                c.user_snapshot = $user_snapshot,
                c.asset_snapshot = $asset_snapshot,
                c.patterns_matched = $patterns_matched,
                c.nodes_consulted = $nodes_consulted
        """, **ctx)
    print(f"  ✓ Created {len(context_nodes)} DecisionContext nodes")
    
    # Create EvolutionEvent nodes
    evolution_nodes = [
        {
            "id": "EVO-0891",
            "event_type": "pattern_confidence",
            "triggered_by": "DEC-7823",
            "before_state": '{"confidence": 0.91, "occurrence_count": 126}',
            "after_state": '{"confidence": 0.94, "occurrence_count": 127}',
            "description": "Pattern PAT-TRAVEL-001 confidence increased based on successful false positive close",
            "timestamp": "2026-01-31T03:48:30Z"
        },
        {
            "id": "EVO-0890",
            "event_type": "threshold_adjustment",
            "triggered_by": "DEC-7819",
            "before_state": '{"auto_close_threshold": 0.88}',
            "after_state": '{"auto_close_threshold": 0.90}',
            "description": "Auto-close threshold for travel alerts adjusted based on sustained accuracy",
            "timestamp": "2026-01-31T07:32:00Z"
        },
        {
            "id": "EVO-0889",
            "event_type": "new_pattern",
            "triggered_by": "DEC-7822",
            "before_state": '{}',
            "after_state": '{"pattern_id": "PAT-PHISH-Q1-2026", "confidence": 0.94}',
            "description": "New phishing campaign pattern identified from Q1 2026 campaign",
            "timestamp": "2026-01-31T08:17:00Z"
        },
    ]
    
    for evo in evolution_nodes:
        session.run("""
            MERGE (e:EvolutionEvent {id: $id})
            SET e.event_type = $event_type,
                e.triggered_by = $triggered_by,
                e.before_state = $before_state,
                e.after_state = $after_state,
                e.description = $description,
                e.timestamp = datetime($timestamp)
        """, **evo)
    print(f"  ✓ Created {len(evolution_nodes)} EvolutionEvent nodes")

driver.close()
print("-" * 50)
print("✓ Decision Trace nodes created")

## Cell 28: Create Decision Trace Relationships (INCLUDING TRIGGERED_EVOLUTION)

In [ ]:
# This creates the relationships that connect decision traces.
# CRITICAL: This includes TRIGGERED_EVOLUTION - THE KEY DIFFERENTIATOR

print("Creating Decision Trace relationships...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # Decision -[:HAD_CONTEXT]-> DecisionContext
    for ctx in context_nodes:
        session.run("""
            MATCH (d:Decision {id: $decision_id})
            MATCH (c:DecisionContext {id: $context_id})
            MERGE (d)-[:HAD_CONTEXT]->(c)
        """, decision_id=ctx["decision_id"], context_id=ctx["id"])
    print("  ✓ Created HAD_CONTEXT relationships")
    
    # Decision -[:FOR_ALERT]-> Alert
    for dec in decision_nodes:
        session.run("""
            MATCH (d:Decision {id: $decision_id})
            MATCH (a:Alert {id: $alert_id})
            MERGE (d)-[:FOR_ALERT]->(a)
        """, decision_id=dec["id"], alert_id=dec["alert_id"])
    print("  ✓ Created FOR_ALERT relationships")
    
    # ═══════════════════════════════════════════════════════════════════════
    # TRIGGERED_EVOLUTION — THE KEY DIFFERENTIATOR
    # This is what makes our demo unique. What SIEMs don't have.
    # ═══════════════════════════════════════════════════════════════════════
    
    triggered_evolutions = [
        {
            "decision_id": "DEC-7823",
            "evolution_id": "EVO-0891",
            "impact": "pattern_confidence_increase",
            "magnitude": 0.03,
        },
        {
            "decision_id": "DEC-7819",
            "evolution_id": "EVO-0890",
            "impact": "threshold_adjustment",
            "magnitude": 0.02,
        },
        {
            "decision_id": "DEC-7822",
            "evolution_id": "EVO-0889",
            "impact": "new_pattern_created",
            "magnitude": 1.0,
        },
    ]
    
    for te in triggered_evolutions:
        session.run("""
            MATCH (d:Decision {id: $decision_id})
            MATCH (e:EvolutionEvent {id: $evolution_id})
            MERGE (d)-[:TRIGGERED_EVOLUTION {
                impact: $impact,
                magnitude: $magnitude,
                timestamp: datetime()
            }]->(e)
        """, **te)
    print(f"  ✓ Created {len(triggered_evolutions)} TRIGGERED_EVOLUTION relationships (THE KEY)")
    
    # CAUSED relationships (causal chain)
    caused_relationships = [
        {
            "from_decision": "DEC-7823",
            "to_decision": "DEC-7819",
            "reason": "Similar travel pattern recognition applied"
        },
        {
            "from_decision": "DEC-7822",
            "to_decision": "DEC-7817",
            "reason": "Same phishing campaign signature"
        },
    ]
    
    for caused in caused_relationships:
        session.run("""
            MATCH (d1:Decision {id: $from_decision})
            MATCH (d2:Decision {id: $to_decision})
            MERGE (d1)-[:CAUSED {
                reason: $reason,
                timestamp: datetime()
            }]->(d2)
        """, **caused)
    print(f"  ✓ Created {len(caused_relationships)} CAUSED relationships")
    
    # PRECEDENT_FOR relationships
    precedent_relationships = [
        {
            "from_decision": "DEC-7823",
            "to_decision": "DEC-7819",
            "similarity": 0.89,
            "cited_in_reasoning": True
        },
        {
            "from_decision": "DEC-7822",
            "to_decision": "DEC-7817",
            "similarity": 0.94,
            "cited_in_reasoning": True
        },
    ]
    
    for prec in precedent_relationships:
        session.run("""
            MATCH (d1:Decision {id: $from_decision})
            MATCH (d2:Decision {id: $to_decision})
            MERGE (d1)-[:PRECEDENT_FOR {
                similarity: $similarity,
                cited_in_reasoning: $cited_in_reasoning
            }]->(d2)
        """, **prec)
    print(f"  ✓ Created {len(precedent_relationships)} PRECEDENT_FOR relationships")

driver.close()
print("-" * 50)
print("✓ Decision Trace relationships created")

## Cell 29: Verify Neo4j Schema

In [ ]:
# This verifies all the critical nodes and relationships are present.

print("Verifying Neo4j schema...")
print("-" * 50)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

with driver.session() as session:
    # Count all node types
    result = session.run("MATCH (n) RETURN labels(n)[0] as label, count(n) as count ORDER BY label")
    print("Node counts:")
    total_nodes = 0
    for record in result:
        print(f"  {record['label']}: {record['count']}")
        total_nodes += record['count']
    print(f"  TOTAL: {total_nodes}")
    
    print()
    
    # Count relationships
    result = session.run("MATCH ()-[r]->() RETURN type(r) as type, count(r) as count ORDER BY type")
    print("Relationship counts:")
    total_rels = 0
    for record in result:
        print(f"  {record['type']}: {record['count']}")
        total_rels += record['count']
    print(f"  TOTAL: {total_rels}")
    
    print()
    
    # Verify TRIGGERED_EVOLUTION specifically
    result = session.run("""
        MATCH (d:Decision)-[r:TRIGGERED_EVOLUTION]->(e:EvolutionEvent)
        RETURN d.id as decision, e.id as evolution, r.impact as impact
    """)
    print("TRIGGERED_EVOLUTION relationships (THE KEY):")
    te_count = 0
    for record in result:
        print(f"  {record['decision']} → {record['evolution']} ({record['impact']})")
        te_count += 1
    
    # Verification summary
    print("-" * 50)
    expected_nodes = 46  # 5+5+4+5+4+3+2+5+5+5+3 (Asset+User+AlertType+Pattern+Playbook+SLA+Travel+Alert+Decision+Context+Evo)
    expected_te = 3
    
    if te_count == expected_te:
        print(f"✓ TRIGGERED_EVOLUTION count correct: {te_count}")
    else:
        print(f"⚠ TRIGGERED_EVOLUTION count: {te_count} (expected {expected_te})")
    
    print(f"✓ Total nodes: {total_nodes}")
    print(f"✓ Total relationships: {total_rels}")

driver.close()
print("-" * 50)
print("✓ Neo4j schema verified")

## Cell 30: Export Environment Configuration

In [ ]:
# This outputs the .env file content you'll need for local development.

print("=" * 70)
print("ENVIRONMENT CONFIGURATION")
print("Copy this to your .env file:")
print("=" * 70)

env_content = f"""
# GCP Configuration
PROJECT_ID={PROJECT_ID}
REGION={REGION}

# BigQuery
BIGQUERY_DATASET={BIGQUERY_DATASET}

# Firestore
FIRESTORE_DATABASE=(default)

# Neo4j Aura
NEO4J_URI={NEO4J_URI}
NEO4J_USER={NEO4J_USER}
NEO4J_PASSWORD={NEO4J_PASSWORD}

# Vertex AI
VERTEX_AI_LOCATION={REGION}
"""

print(env_content)
print("=" * 70)

## Cell 31: Final Summary

In [ ]:
# This shows everything that was created.

print("=" * 80)
print("                         SETUP COMPLETE!")
print("=" * 80)
print()
print("What was created:")
print("-" * 40)
print()
print(f"BigQuery ({BIGQUERY_DATASET} dataset):")
print("  * metric_contracts (10 SOC metrics with definitions)")
print("  * soc_metrics (4 weeks of MTTR/MTTD/auto-close data)")
print("  * detection_rules (5 rules with FP rates)")
print("  * rule_sprawl_registry (1 duplicate rule for demo)")
print()
print("Firestore (Native mode):")
print("  * assets (5 devices/servers)")
print("  * users (5 users with risk scores)")
print("  * playbooks (4 response procedures)")
print("  * patterns (5 learned attack patterns)")
print("  * alerts (5 security alerts for demo)")
print("  * deployments (2 agent versions: active + canary)")
print("  * travel (2 travel records for context)")
print("  * decisions (2 sample decisions)")
print()
print("Neo4j Aura (security graph):")
print(f"  * ~{total_nodes} nodes (Assets, Users, Alerts, Patterns, etc.)")
print(f"  * ~{total_rels} relationships")
print("  * Including: SUBJECT_TO (Asset→SLA), MATCHES (Alert→Pattern)")
print()
print("Decision Trace Schema (THE KEY DIFFERENTIATOR):")
print("  * 5 Decision nodes")
print("  * 5 DecisionContext nodes")
print("  * 3 EvolutionEvent nodes")
print("  * 3 TRIGGERED_EVOLUTION relationships  <-- THE KEY ADDITION")
print("  * 2 CAUSED relationships (causal chains)")
print("  * 2 PRECEDENT_FOR relationships (precedent matching)")
print()
print("=" * 80)
print()
print("NEXT STEPS:")
print("-" * 40)
print("1. Copy the .env content above to your local project")
print("2. Run: cd ~/projects/soc-copilot-demo")
print("3. Run: claude")
print("4. Ask Claude to start building Tab 2 (Runtime Evolution)")
print()
print("Key file references:")
print("  * CLAUDE.md → CLAUDE_ciso_v1.md")
print("  * Build spec → vc_demo_build_spec_ciso_v1.md")
print("  * Setup notebook → ciso_setup_notebook_v1.py (THIS FILE)")
print()
print("=" * 80)
print("                    SOC Copilot Demo Ready!")
print("       Key differentiator: TRIGGERED_EVOLUTION relationships")
print("=" * 80)